In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import ticker
import awkward as ak
import uproot 
import mplhep as hep
import hist
import os
import time
from numba import njit
from matplotlib.colors import LogNorm
import tqdm
import pickle 
from numba import jit



from UtilityAndConst import *
from Functions import *

dir = "/Users/francescolarovere/Documents/Tesi-Triennale/Dati"
os.chdir(dir)

hep.style.use("CMS")
np.set_printoptions(threshold=np.inf)

In [2]:
files = [
    "./ntupleStubsBmtfGmt_LS133.root",
    "./ntupleStubsBmtfGmt_LS263.root",
    "./ntupleStubsBmtfGmt_LS264.root",
    "./ntupleStubsBmtfGmt_LS306.root",
]

stub_bmtf_gmt_features = [
    'orbit', 'bx', 'nStubs', 'stubHwPhi', 'stubHwPhiB', 'stubHwQual', 'stubHwEta', 'stubHwQEta', 'stubStation', 'stubWheel', 'stubSector', 'stubTag',
    'nBmtfMuons', 'bmtfHwPt', 'bmtfHwPtu', 'bmtfHwPhi', 'bmtfHwEta', 'bmtfPt', 'bmtfPtu', 'bmtfPhi', 'bmtfEta', 'bmtfHwSign', 'bmtfHwSignValid', 'bmtfHwDXY', 'bmtfHwQual', 'bmtfProcessor', 
    'nGmtMuons', 'gmtHwPt', 'gmtHwPtu', 'gmtHwPhi', 'gmtHwEta', 'gmtHwPhiAtVtx', 'gmtHwEtaAtVtx', 'gmtPt', 'gmtPtu', 'gmtPhi', 'gmtEta', 'gmtPhiAtVtx', 'gmtEtaAtVtx', 'gmtCharge', 'gmtDxy', 'gmtQual', 'gmtTfIndex']

stub_bmtf_gmt_list = []
for file in files:
    with uproot.open(file) as f:
        evs = f["bmtfNtuplizer/Events"]
        temp_awk = evs.arrays(stub_bmtf_gmt_features, library="ak")
        stub_bmtf_gmt_list.append(temp_awk)
        
bmtf_gmt = ak.concatenate(stub_bmtf_gmt_list, axis=0)

In [3]:
#-----------------------------------------------Stubs------------------------------------------------
nStubs = bmtf_gmt.nStubs
stubsStation = bmtf_gmt.stubStation
stubsSector = bmtf_gmt.stubSector
stubsWheel = bmtf_gmt.stubWheel
stubsPhiB = bmtf_gmt.stubHwPhiB


#-----------------------------------------------BMTF------------------------------------------------
nBmtf = bmtf_gmt.nBmtfMuons
BmtfPhi = bmtf_gmt.bmtfPhi
BmtfEta = bmtf_gmt.bmtfEta
BmtfPt = bmtf_gmt.bmtfPt
BmtfQual = bmtf_gmt.bmtfHwQual
bmtfProc = bmtf_gmt.bmtfProcessor

#-----------------------------------------------GMT------------------------------------------------
nGmt = bmtf_gmt.nGmtMuons
GmtPhi = bmtf_gmt.gmtPhi
GmtEta = bmtf_gmt.gmtEta
GmtPt = bmtf_gmt.gmtPt
GmtQual = bmtf_gmt.gmtQual


In [4]:
'''for i in range(1000):
  if nStubs[i] > 4:
    print(i, nStubs[i], stubsStation[i], stubsSector[i], stubsWheel[i])
    print(i, nBmtf[i], bmtfProc[i])'''

'for i in range(1000):\n  if nStubs[i] > 4:\n    print(i, nStubs[i], stubsStation[i], stubsSector[i], stubsWheel[i])\n    print(i, nBmtf[i], bmtfProc[i])'

In [5]:
BmtfFeatures = [nBmtf, BmtfPhi, BmtfEta, BmtfPt, BmtfQual]
GmtFeatures = [nGmt, GmtPhi, GmtEta, GmtPt, GmtQual]

In [18]:
def DeltaR(Phi1, Phi2, Eta1, Eta2):
   return np.sqrt((Phi1 - Phi2)**2 + (Eta1 - Eta2)**2)

def HSCP(Features, Number):
   MuonsEvents = []

   for i in tqdm.tqdm(range(Number - 1)):
      nMuons = Features[0][i]
      nMuonsP1 = Features[0][i + 1]
      ArrPhi, ArrEta = np.array(Features[1][i]), np.array(Features[2][i])
      ArrPt = np.array(Features[3][i])
      ArrQual = np.array(Features[4][i])

      DeltaRList, Index = [], []

      PtDiff, matched = [], []

      if nMuons == 0:
         matched.append(False)

      else:
         if nMuonsP1 == 0:
            matched.append(False)

        
         elif nMuons == 2 and nMuonsP1 == 1:
            for j in range(nMuons):
               PtDiff.append(np.abs(ArrPt[j] - Features[3][i + 1][0]))

            j = np.argmin(PtDiff)

            TT = [j == idx for idx in range(nMuons)]
            DeltaRTemp = DeltaR(ArrPhi[j], BmtfFeatures[1][i + 1][0], ArrEta[j], BmtfFeatures[2][i + 1][0])
            DeltaRList.append(DeltaRTemp)
            Index.append((j, 0)) 
            matched.extend(TT)

         else:
            for j in range(nMuons):
               DeltaRTemp = DeltaR(ArrPhi[j], BmtfFeatures[1][i + 1][:], ArrEta[j], BmtfFeatures[2][i + 1][:])
               minDeltaR = np.min(DeltaRTemp)
               IndexP1 = np.argmin(DeltaRTemp)
               DeltaRList.append(minDeltaR)
               Index.append((j, IndexP1)) 
               matched.append(True)


      MuonProperty = {}
      MuonP1Property = {}
      
      for feature in ['Pt', 'Eta', 'Phi', 'Qual']:
         MuonProperty[feature] = []
         MuonP1Property[feature] = []

      for indx, (j, IndexP1) in enumerate(Index):
         MuonProperty['Pt'].append(ArrPt[j])
         MuonProperty['Eta'].append(ArrEta[j])
         MuonProperty['Phi'].append(ArrPhi[j])
         MuonProperty['Qual'].append(ArrQual[j])

         MuonP1Property['Pt'].append(BmtfFeatures[3][i + 1][IndexP1])
         MuonP1Property['Eta'].append(BmtfFeatures[2][i + 1][IndexP1])
         MuonP1Property['Phi'].append(BmtfFeatures[1][i + 1][IndexP1])
         MuonP1Property['Qual'].append(BmtfFeatures[4][i + 1][IndexP1])
         

    #Salvataggio nel dizionario
      Events = {
         'event': i,
         'nMuons': nMuons,
         'nMuonsP1': nMuonsP1,
         'matched': matched,
         'DeltaR': DeltaRList,
         'MuonProperty': MuonProperty,
         'MuonP1Property': MuonP1Property

      }

      MuonsEvents.append(Events)

   return ak.Array(MuonsEvents)


In [19]:
MuonsEventsBMTF = HSCP(BmtfFeatures, Number = 1_000_000)#len(BmtfFeatures[0]))

100%|██████████| 99999/99999 [00:36<00:00, 2764.85it/s]


In [80]:
'''#Save Dictionary
with open('MuonsEventsBMTF2.pkl', 'wb') as f:
    pickle.dump(MuonsEventsBMTF, f)'''

In [81]:
'''with open('MuonsEventsBMTF2.pkl', 'rb') as f:
    MuonsEventsBMTF = pickle.load(f)'''

In [20]:
MuonsEventsBMTF[280]

<Record {event: 280, nMuons: 2, ...} type='{event: int64, nMuons: int64, nM...'>

In [ ]:
bins = np.arange(0, 7, 0.1)
DeltaRDist_HistBMTF = hist.Hist(hist.axis.Variable(bins, name="DeltaR", label="DeltaR"))
DeltaRDist_HistGMT = hist.Hist(hist.axis.Variable(bins, name="DeltaR", label="DeltaR"))

DeltaRDist_HistBMTF.fill(DeltaR = ak.flatten(MuonsEventsBMTF.DeltaR))
#DeltaRDist_HistGMT.fill(DeltaR = ak.flatten(MuonsEventsGMT.DeltaR))

fig, ax = plt.subplots(figsize=FIGSIZE, constrained_layout=True)
set_label_font(ax, FONTSIZE)
set_tick_font(ax, FONTSIZE-2)
draw_grid(ax, axis="y")
draw_cms_label(ax, fontsize= FONTSIZE)

hep.histplot(DeltaRDist_HistBMTF, yerr = False)
hep.histplot(DeltaRDist_HistGMT, yerr = False)

ax.grid(True)
ax.legend(fontsize = FONTSIZE)
ax.set_ylabel("Events")
#ax.set_yscale('log')

ax.tick_params(axis='both', which='both', labelsize = FONTSIZE - 5)

plt.show()


In [128]:
def FilteredMuonsEvents(MuonsEvents, Threshold):

   MatchedMuonsEvents = MuonsEvents[MuonsEventsBMTF.matched == True]

   MuonsDeltaR = MatchedMuonsEvents['DeltaR']
   MuonsPt = MatchedMuonsEvents['MuonProperty']['Pt']
   MuonsPtP1 = MatchedMuonsEvents['MuonP1Property']['Pt']
   MuonsPhi = MatchedMuonsEvents['MuonProperty']['Phi']
   MuonPhiP1 = MatchedMuonsEvents['MuonP1Property']['Phi']

   matched_copy = np.array(MatchedMuonsEvents['matched'].to_list())

   matched_copy[:] = False

   for i in tqdm.tqdm(range(len(MatchedMuonsEvents))):
      for j in range(len(MatchedMuonsEvents['DeltaR'][i])):

         if ((MuonsDeltaR[i][j] < Threshold[0]) and (MuonsPt[i][j] > Threshold[1] and MuonsPtP1[i][j] > Threshold[1]) and (MuonsPhi[i][j] < Threshold[2] and MuonPhiP1[i][j] < Threshold[2])):
            matched_copy[i] = True
   MatchedMuonsEvents['matched'] = ak.from_numpy(matched_copy)

   return MatchedMuonsEvents

In [ ]:
FilMuonsEvents1 = FilteredMuonsEvents(MuonsEventsBMTF, [2, 0, 0])

In [ ]:
FinalMuonsEvents1 = FilMuonsEvents1[FilMuonsEvents1.matched == True]
FinalMuonsEvents1

In [144]:
Pt1 = FinalMuonsEvents1['MuonProperty']['Pt']
PtP11 = FinalMuonsEvents1['MuonP1Property']['Pt']
Eta1 = FinalMuonsEvents1['MuonProperty']['Eta']
EtaP11 = FinalMuonsEvents1['MuonP1Property']['Eta']

In [ ]:
bins = np.arange(0, 255, 1)
PtDist_HistBMTF1 = hist.Hist(hist.axis.Variable(bins, name="Pt"))
PtP1Dist_HistBMTF1 = hist.Hist(hist.axis.Variable(bins, name="PtP1"))

PtDist_HistBMTF1.fill(Pt = ak.flatten(Pt1))
PtP1Dist_HistBMTF1.fill(PtP1 = ak.flatten(PtP11))

fig, ax = plt.subplots(figsize=FIGSIZE, constrained_layout=True)
set_label_font(ax, FONTSIZE)
set_tick_font(ax, FONTSIZE-2)
draw_grid(ax, axis="y")
draw_cms_label(ax, fontsize= FONTSIZE)

hep.histplot(PtDist_HistBMTF1, yerr = False, label="Pt1")
#hep.histplot(PtP1Dist_HistBMTF1, yerr = False, label="PtP1")


ax.grid(True)
ax.legend(fontsize = FONTSIZE)
ax.set_ylabel("Events")
ax.set_yscale('log')

ax.legend()

ax.tick_params(axis='both', which='both', labelsize = FONTSIZE - 5)

plt.show()